In [47]:
import streamlit as st
import pandas as pd
import numpy as np
import geopandas as gpd  # For handling GeoJSON data
import folium
import branca.colormap as cm
from streamlit_folium import folium_static  # Import to render folium maps in Streamlit
from sklearn.metrics import accuracy_score

In [48]:
# Load the GeoJSON file
#zip_geojson = gpd.read_file('select_zips.geojson')

zip_geojson = gpd.read_file('demo_zips.geojson')

# Load person data, forcing ZIP to be read as strings
person_data = pd.read_csv('data.csv', dtype={'ZIP': str})

# Ensure ZIP codes have leading zeros and handle floats
person_data['ZIP'] = person_data['ZIP'].apply(lambda x: str(int(float(x))).zfill(5) if pd.notnull(x) else '')

# Ensure GeoJSON ZIP codes are formatted as strings with leading zeros
zip_geojson['ZIP'] = zip_geojson['ZCTA5CE10'].astype(str).str.zfill(5)

# Check if ZIPs were properly converted
print(person_data['ZIP'].head())
print(zip_geojson['ZIP'].head())


0    04265
1    33467
2    02131
3    17968
4    94118
Name: ZIP, dtype: object
0    02457
1    02458
2    02459
3    02460
4    02461
Name: ZIP, dtype: object


In [49]:
# Function to calculate metrics for each ZIP
def calculate_zip_metrics(stats_data_person, fico_cutoff, energyscore_cutoff):
    # Create masks for conditions
    stats_data_person['FICO_PASS'] = stats_data_person['FICO_V9_SCORE'] > fico_cutoff
    stats_data_person['ENERGYSCORE_PASS'] = stats_data_person['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff

    # Function to calculate various metrics
    def calc_metrics(group):
        total_population = len(group)
        if total_population == 0:
            return pd.Series({
                'Total Population': 0,
                'Percent Below FICO': 0,
                'Percent Above FICO': 0,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })
        
        
        below_fico = group[group['FICO_PASS'] == False]
        above_fico = group[group['FICO_PASS'] == True]

        if len(below_fico) == 0:
            return pd.Series({
                'Total Population': total_population,
                'Percent Below FICO': 0,
                'Percent Above FICO': len(above_fico) / total_population,
                'FICO Accuracy': np.nan,
                'EnergyScore Accuracy': np.nan,
                'Qualification Increase': 0,
            })

        below_fico_pass = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] <= energyscore_cutoff]
        below_fico_fail = below_fico[below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff]

        pct_below_fico = len(below_fico) / total_population
        pct_above_fico = len(above_fico) / total_population

        if len(below_fico_pass) == 0:
            percent_increase_in_qualifications = 0
        else:
            percent_increase_in_qualifications = (len(below_fico_pass) / total_population) * 100

        # get FICO accuracy, precision, recall, f1 and roc_auc score
       # fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['WEIGHTED_ENERGYSCORE'] > energyscore_cutoff) if len(below_fico) > 0 else np.nan
        fico_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['FICO_PASS']) if len(below_fico) > 0 else np.nan

        energy_accuracy = accuracy_score(below_fico['WEIGHTED_ACTUAL_OUTPUT'], below_fico['ENERGYSCORE_PASS']) if len(below_fico) > 0 else np.nan

      #  accuracy_increase = energy_accuracy - fico_accuracy

        return pd.Series({
            'Total Population': total_population,
            'Percent Below FICO': pct_below_fico,
            'Percent Above FICO': pct_above_fico,
            'FICO Accuracy': fico_accuracy,
            'EnergyScore Accuracy': energy_accuracy,
            'Qualification Increase': percent_increase_in_qualifications,
          #  'Accuracy Percentage Increase': accuracy_increase,

           # 'Accuracy Percentage Increase': (energy_accuracy - fico_accuracy) / fico_accuracy * 100 if fico_accuracy * energy_accuracy> 0 else 0,
           

        })

    # Group by ZIP and apply metrics calculation
    zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)

    zip_metrics = zip_metrics.reset_index()
    return zip_metrics

fico_threshold = 700
energy_score_threshold = 0.5
# Calculate metrics and merge with geo data
zip_metrics = calculate_zip_metrics(person_data, fico_threshold, energy_score_threshold)
zip_level_geo = pd.merge(zip_metrics, zip_geojson, on='ZIP', how='left')
zip_level_geo = zip_level_geo.dropna(subset=['geometry'])
zip_level_geo = gpd.GeoDataFrame(zip_level_geo, geometry='geometry')


/var/folders/4_/36zlcy3x3m12nwb2nfhmkkjc0000gp/T/ipykernel_37470/1243698195.py:68: DeprecationWarning: DataFrameGroupBy.apply operated on the grouping columns. This behavior is deprecated, and in a future version of pandas the grouping columns will be excluded from the operation. Either pass `include_groups=False` to exclude the groupings or explicitly select the grouping columns after groupby to silence this warning.
  zip_metrics = stats_data_person.groupby('ZIP').apply(calc_metrics)


In [ ]:
zip_level_geo

In [ ]:


def load_state_util(state_name):
    if state_name == 'New Mexico':
        temp = gpd.read_file('nm_utils.geojson')
        temp = temp[['new_name', 'geometry']]
        return temp
    elif state_name == 'Massachusetts':
        return gpd.read_file('ma_utils.geojson')


def calculate_zip_to_util(zip_level_geo, util_name, state_name):
    # Load the utility data for the state
    state_util = load_state_util(state_name)

    # Ensure ZIP code geometries have the same projection as the utility data
    zip_level_geo = zip_level_geo.to_crs(state_util.crs)

    # Convert the ZIP geometries to representative points (instead of centroids)
    zip_level_geo['geometry'] = zip_level_geo.representative_point()

    # Perform spatial join with utility data based on point locations using predicate
    zip_level_geo = gpd.sjoin(zip_level_geo, state_util, how='left', predicate='within')

    # Group by utility name ('new_name') and calculate the mean of 'Qualification Increase'
    zip_to_util = zip_level_geo.groupby('new_name')['Qualification Increase'].mean().reset_index()

    return zip_to_util

# Example usage
state_name = 'New Mexico'
util_name = 'Xcel'
zip_to_util = calculate_zip_to_util(zip_level_geo, util_name, state_name)
zip_to_util.head()


In [ ]:
zip_metrics

In [ ]:
zip_level_geo.shape

In [ ]:
zip_metrics['FICO Accuracy'].mean(),zip_metrics['EnergyScore Accuracy'].mean()

In [ ]:
zip_geojson.head()